# LASSO Regression: Implementation with Coordinate Descent
In this notebook, you will implement your very own LASSO solver via coordinate descent. You will:
* Write a function to normalize features
* Implement coordinate descent for LASSO
* Explore the effects of L1 penalty

In [120]:
import numpy as np
import pandas as pd
np.set_printoptions(precision=3,suppress=True)

## Load in house sales data

Dataset is from house sales in King County.

In [121]:
sales = pd.read_csv('home_data.gz')

## Import useful functions from previous notebook

As in a previous quiz, we convert the DataFrame into a 2D Numpy array. Copy and paste your code for `pandas2numpy` here:

In [122]:
def pandas2numpy(dataframe, features, target):
    sub_dataframe = dataframe[features]
    y = dataframe[target]
    sub_dataframe.insert(0, 'bias', 1)
    X = sub_dataframe.to_numpy()
    return X, y

Also, copy and paste the `predict_target()` function to compute the predictions for an entire matrix of features given the matrix and the weights:

In [123]:
def predict_target(design_matrix, model):
    return design_matrix@model

## Normalize features
In the house dataset, features vary wildly in their relative magnitude: `sqft_living` is very large overall compared to `bedrooms`, for instance. As a result, weight for `sqft_living` would be much smaller than weight for `bedrooms`. This is problematic because "small" weights are dropped first as `l1_penalty` goes up. 

To give equal considerations for all features, we need to **normalize features**. As discussed in class, we divide each feature  by its 2-norm so that the transformed feature has norm 1. 

We can do this normalization easily with NumPy. First, consider a small matrix.

In [124]:
X = np.array([[3.,5.,8.],[4.,12.,15.]])
print(X)

***Question 1.*** Using a NumPy command, compute and print the 2-norms of each column of X. Then normalize and print X.
Your output should be: $$[ 5., 13., 17.]$$ and
$$
\begin{bmatrix}
0.6 & 0.38 & 0.47 \\
0.8 & 0.92 & 0.88
\end{bmatrix}
$$


In [125]:
# YOUR CODE HERE
norm = np.linalg.norm(X, axis = 0)
X_norm = X / norm
print(norm)
print(X_norm)

***Question 2.*** Using the shorthand we just covered, write a short function called `normalize_features(feature_matrix)`, which normalizes columns of a given feature matrix. The function should return a pair `(normalized_features, norms)`, where the second item contains the norms of original features. As discussed class, we will use these norms to normalize the test data in the same way as we normalized the training data. 

In [126]:
# YOUR CODE HERE
def normalize_features(feature_matrix):
    norm = np.linalg.norm(feature_matrix, axis = 0)
    feature_matrix_norm = feature_matrix / norm
    return feature_matrix_norm, norm

To test the function, run the following:

In [127]:
features, norms = normalize_features(np.array([[3.,6.,9.],[4.,8.,12.]]))
print(features)
# should print
# [[ 0.6  0.6  0.6]
#  [ 0.8  0.8  0.8]]
print(norms)
# should print
# [5.  10.  15.]

# Implementing Coordinate Descent with normalized features

We seek to obtain a sparse set of weights by minimizing the LASSO cost function

$$\sum (\hat{y}_i-y_i)^2 + \lambda (|w_1| + |w_2| +\cdots |w_d|)$$

(By convention, we do not include $w_0$ in the L1 penalty term as we never want to push the intercept to zero.)

The absolute value sign makes the cost function non-differentiable, so simple gradient descent is not viable. Instead, we will use **coordinate descent**: at each iteration, we will fix all weights but weight `i` and find the value of $w_i$ that minimizes the objective. That is, we look for
$$\mathrm{argmin}_{w_i}\left( \sum (\mathbf{w}\cdot\mathbf{x}_i-y_i)^2 + \lambda (|w_1| + |w_2| +\cdots |w_d|)\right) $$

where all weights other than $w_i$ are held constant. We will optimize one $w_i$ at a time, circling through the weights multiple times, until convergence is reached.

For this notebook, we use **cyclical coordinate descent with normalized features**, where we cycle through coordinates 0 to $d$  in order, and assume that the features were normalized as discussed above. The formula for optimizing each weight, other than the intercept, is as follows:

$$ w_j = \left\{
\begin{array}{cl}
r_j + \lambda/2 & \text{if } r_j < -\lambda/2 \\
0 & \text{if } r_j\in[-\lambda/2,\lambda/2] \\
r_j - \lambda/2 & \text{if } r_j > \lambda/2
\end{array}
\right. $$
where
$$r_j = \sum_{i=1}^N x_{ij}\left(y_i - \sum_{k\ne j}w_kx_{ik}\right) $$
applied to the normalized features.

Recall that we do not regularize the weight of the constant feature (intercept) $w_0$, so, for this weight, the update is simply $w_0 = r_0$.

## Effect of L1 penalty

Let us consider a simple model with 2 features:

In [128]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
simple_feature_matrix, output = pandas2numpy(sales, simple_features, my_output)

Don't forget to normalize features:

In [129]:
simple_feature_matrix, norms = normalize_features(simple_feature_matrix)

We assign some random set of initial weights and inspect the values of `ro[i]`:

In [130]:
weights = np.array([1., 4., 1.])

Use `predict_target()` to make predictions on this data.

In [131]:
prediction = predict_target(simple_feature_matrix,weights)

***Question 3.*** Compute the values of $r_j$ for each feature in this simple model, using the formula given above.

In [132]:
# YOUR CODE HERE
def get_rj(X, y, w, j):
    N = X.shape[0]
    rj = 0
    for i in range(N):
        pred_wj = sum(w[k] * X[i, k] for k in range(X.shape[1]) if k != j)
        rj += X[i, j] * (y[i] - pred_wj)
    return rj

rjs = [get_rj(simple_feature_matrix, output, weights, j) for j in range(1, simple_feature_matrix.shape[1])]

print(rjs)


***Question 4.***
Recall that, whenever $r_i$ falls between `-l1_penalty/2` and `l1_penalty/2`, the corresponding weight $w_i$ is sent to zero. Now suppose we were to take one step of coordinate descent on either feature 1 or feature 2. What range of values of `l1_penalty` **would not** set $w_1$ to zero, but **would** set $w_2$ to zero, if we were to take a step in that coordinate? 

In [133]:
# YOUR CODE HERE
l1_penalty_min = -2 * rjs[1]
l1_penalty_max = 2 * rjs[1]
print((l1_penalty_min, l1_penalty_max))

***Question 5.*** 
What range of values of `l1_penalty` would set **both** $w_1$ and $w_2$ to zero, if we were to take a step in that coordinate? 

In [134]:
# YOUR CODE HERE
l1_penalty_min1 = -2 * rjs[0]
l1_penalty_max1 = 2 * rjs[0]
l1_penalty_min2 = -2 * rjs[1]
l1_penalty_max2 = 2 * rjs[1]
print((l1_penalty_min1, l1_penalty_max1))
print((l1_penalty_min2, l1_penalty_max2))

So we can say that $r_j$ quantifies the significance of the $j$-th feature: the larger $r_j$ is, the more likely it is for the $j$-th feature to be retained.

## Single Coordinate Descent Step

***Question 6.*** Using the formula above, implement coordinate descent to minimize the cost function over a single feature $i$. Note that the intercept (weight 0) is not regularized. The function should accept the index $i$ of the feature to optimize over, the feature matrix, the output, the current weights, and the l1 penalty. The function should return new weight for feature $i$.

In [135]:
# YOUR CODE HERE
def lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty):
    prediction = predict_target(feature_matrix, weights)
    x = feature_matrix[:, i]
    r = x.T @ (output - prediction + x * weights[i])
    if i == 0:
        w_i = r
    elif r < (-l1_penalty/2):
        w_i = r + l1_penalty/2
    elif r > (l1_penalty/2):
        w_i = r - l1_penalty/2
    else:
        w_i = 0
    return w_i

To test the function, run the following cell:

In [136]:
# should print 0.425558846691
import math
new1 = lasso_coordinate_descent_step(1, np.array([[3./math.sqrt(13),1./math.sqrt(10)],[2./math.sqrt(13),3./math.sqrt(10)]]), 
                                    np.array([1., 1.]), np.array([1., 4.]), 0.1)
print(new1)

## Cyclical coordinate descent 

Now that we have a function that optimizes the cost function over a single coordinate, let us implement cyclical coordinate descent where we optimize coordinates 0, 1, ..., $d$ in order and repeat.

When do we know to stop? Each time we scan all the coordinates (features) once, we measure the change in weight for each coordinate. If no coordinate changes by more than a specified threshold, we stop.

For each iteration:
1. As you loop over features in order and perform coordinate descent, measure how much each coordinate changes.
2. After the loop, if the maximum change across all coordinates falls below the tolerance, stop. Otherwise, go back to step 1.

Return weights

**IMPORTANT: when computing a new weight for coordinate i, make sure to incorporate the new weights for coordinates 0, 1, ..., i-1. One good way is to update your weights variable in-place. See following pseudocode for illustration.**

***Question 7.*** Complete the function `lasso_cyclical_coordinate_descent(feature_matrix, output, initial_weights, l1_penalty, tolerance)`. The meaning of the arguments is the same as above.

In [137]:
# YOUR CODE HERE
def lasso_cyclical_coordinate_descent(feature_matrix, output, initial_weights, l1_penalty, tolerance):
    c = tolerance + 1
    w = initial_weights

    while c > tolerance:
        c = 0

        for i in range(len(initial_weights)):
            ow = w[i]
            w_i = lasso_coordinate_descent_step(i, feature_matrix, output, w, l1_penalty)
            w[i] = w_i
            
            if abs(w_i - ow) > c:
                c = abs(w_i - ow)

    return w

Let's take our function for a spin on the sales data set, using the following parameters.

In [138]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
initial_weights = np.zeros(3)
l1_penalty = 1e7
tolerance = 1.0

***Question 8.*** First, create a normalized version of the feature matrix, say `normalized_simple_feature_matrix`. Then, run your implementation of lasso coordinate descent.
1. What is the MSE of the learned model on the normalized dataset? (Hint: use the normalized feature matrix when you make predictions.)
2. Which features had weight zero at convergence?

In [139]:
# YOUR CODE HERE
simple_feature_matrix, output = pandas2numpy(sales, simple_features, my_output)
normalized_simple_feature_matrix, norms = normalize_features(simple_feature_matrix)
weights = lasso_cyclical_coordinate_descent(normalized_simple_feature_matrix, output, initial_weights, l1_penalty, tolerance)

mse = np.mean((output - predict_target(normalized_simple_feature_matrix, weights))**2)

print(f'MSE: {mse}')
print(f'Int: {weights[0]}')
for feature in simple_features:
    print(f'{feature}: {weights[1 + simple_features.index(feature)]}')

## Evaluating LASSO fit with more features

Let us split the sales dataset into training and test sets.

In [140]:
train_data = sales.sample(frac=0.8, random_state=0)
test_data = sales.drop(train_data.index)

Let us consider the following set of features.

In [141]:
all_features = ['bedrooms',
                'bathrooms',
                'sqft_living',
                'sqft_lot',
                'floors',
                'waterfront', 
                'view', 
                'condition', 
                'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 
                'yr_renovated']

First, create a normalized feature matrix from the training data with these features.  (Make sure to store the norms for the normalization, since we'll use them later)

In [142]:
my_output = 'price'
initial_weights = np.zeros(1+len(all_features))
l1_penalty = 1e7
tolerance = 1.0
(all_feature_matrix, output) = pandas2numpy(train_data, all_features, my_output)
(normalized_all_feature_matrix, all_norms) = normalize_features(all_feature_matrix)

***Question 9.*** First, learn the weights with `l1_penalty=1e7`, on the training data. Initialize weights to all zeros, and set the `tolerance=1`, as suggested above.  Call resulting weights `weights1e7`, you will need them later. What features had non-zero weight in this case?

In [143]:
# YOUR CODE HERE
weights1e7 = lasso_cyclical_coordinate_descent(normalized_all_feature_matrix, output, initial_weights, l1_penalty, tolerance)

print(f'Int: {weights1e7[0]}')
print([(f'{feature}: {weights1e7[1 + all_features.index(feature)]}') for feature in all_features])

***Question 10.*** Next, learn the weights with `l1_penalty=1e8`, on the training data. Initialize weights to all zeros, and set the `tolerance=1`.  Call resulting weights `weights1e8`, you will need them later. What features had non-zero weight in this case?

In [144]:
l1_penalty = 1e8
initial_weights = np.zeros(1+len(all_features))
# YOUR CODE HERE
weights1e8 = lasso_cyclical_coordinate_descent(normalized_all_feature_matrix, output, initial_weights, l1_penalty, tolerance)

print(f'Int: {weights1e8[0]}')
print([(f'{feature}: {weights1e8[1 + all_features.index(feature)]}') for feature in all_features])

***Question 11.*** Finally, learn the weights with `l1_penalty=1e4`, on the training data. Initialize weights to all zeros, and set the `tolerance=5e5`.  Call resulting weights `weights1e4`, you will need them later. What features had non-zero weight in this case? (*Warning: This case may take quite a bit longer to converge than the ones above.*)

In [145]:
l1_penalty = 1e4
initial_weights = np.zeros(1+len(all_features))
# YOUR CODE HERE
tolerance = 5e5
weights1e4 = lasso_cyclical_coordinate_descent(normalized_all_feature_matrix, output, initial_weights, l1_penalty, tolerance)

print(f'Int: {weights1e4[0]}')
print([(f'{feature}: {weights1e4[1 + all_features.index(feature)]}') for feature in all_features])

## Rescaling learned weights

Recall that we normalized our feature matrix, before learning the weights.  To use these weights on a test set, we would need to normalize the test data in the same way.

Alternatively, we can rescale the learned weights, so we never have to worry about normalizing the test data: 

In this case, we must scale the resulting weights so that we can make predictions with *original* feature values:
 1. Store the norms of the original features to a vector called `norms`:
```
features, norms = normalize_features(features)
```
 2. Run Lasso on the normalized features and obtain a `weights` vector
 3. Compute the weights for the original features and stored them in `weights_normalized` 

Now, we can apply `weights_normalized` to the test data, without normalizing it first!

***Question 12.*** Create a normalized version of each of the weights learned above: (`weights1e4`, `weights1e7`, `weights1e8`).

To check your results, if you call `normalized_weights1e7` the normalized version of `weights1e7`, then:
```
print normalized_weights1e7[3]
```
should return 161.31745624837794.

In [146]:
# YOUR CODE HERE
normalized_weights1e4 = weights1e4 / all_norms
normalized_weights1e7 = weights1e7 / all_norms
normalized_weights1e8 = weights1e8 / all_norms

print(normalized_weights1e7[3])

## Evaluating each of the learned models on the test data

Let's now evaluate the three models on the test data:

In [147]:
test_feature_matrix, test_output = pandas2numpy(test_data, all_features, 'price')

***Question 13.*** Compute the MSE of each of the three normalized weights on the (unnormalized) `test_feature_matrix`. Print the results. Which model performed best on the test data?

In [148]:
# YOUR CODE HERE
mse_1 = np.mean((test_output - predict_target(test_feature_matrix, normalized_weights1e4))**2)
mse_2 = np.mean((test_output - predict_target(test_feature_matrix, normalized_weights1e7))**2)
mse_3 = np.mean((test_output - predict_target(test_feature_matrix, normalized_weights1e8))**2)

print(f'1e4: {mse_1}') # performed the best on test data
print(f'1e7: {mse_2}')
print(f'1e8: {mse_3}')

1e4 performed best